In [21]:
from src.data_reader import DataReader
from src.constant import *
import matplotlib.pyplot as plt
import numpy as np
from src.datasets.dataset import Dataset
from src.models.itemknn import ItemKNN
import pandas as pd

In [2]:
dataset = Dataset()

In [3]:
split_dict = dataset.get_split()

In [4]:
train, train_label = split_dict[TRAIN]

In [14]:
val, val_label = split_dict[VAL]

In [9]:
lead_data = dataset.get_test_leaderboard_sessions()

In [5]:
model = ItemKNN(dataset, topk=1000)

In [6]:
model.compute_similarity_matrix(train)

Done: 100%|██████████| 23691/23691 [00:00<00:00, 31890.66it/s]


In [15]:
recs = model.recommend(
    interactions=val,
    remove_seen=True,
    cutoff=100,
)

100%|██████████| 1/1 [00:46<00:00, 46.85s/it]


In [16]:
recs

,session_id,item_id,score,rank
0,107,6258,0.026263,1
1,107,17928,0.021998,2
2,107,17787,0.021563,3
3,107,12940,0.019449,4
4,107,14480,0.019189,5
...,...,...,...,...
9999995,4440001,19161,0.029202,96
9999996,4440001,16935,0.029180,97
9999997,4440001,15784,0.028889,98
9999998,4440001,19194,0.028785,99


In [17]:
val_label

,session_id,item_id,date
0,107,3370,2020-01-15 06:07:23.177
1,108,15639,2020-06-06 17:25:15.508
2,124,3126,2020-05-18 11:04:15.420
3,171,11806,2020-12-18 01:12:30.945
4,324,22049,2020-05-25 15:40:17.987
...,...,...,...
99995,4439892,7623,2020-03-01 13:52:26.972
99996,4439953,2179,2021-04-02 11:31:29.357
99997,4439964,17216,2020-03-24 08:18:38.953
99998,4439986,2445,2021-05-13 11:56:37.464


In [18]:
recs = recs.set_index([SESS_ID, ITEM_ID])
val = val_label.set_index([SESS_ID, ITEM_ID])

In [20]:
recs = recs.reset_index()

In [25]:

merged=pd.merge(recs, val, on=[SESS_ID, ITEM_ID], how="right")

In [30]:
merged["mrr"] = 1/merged["rank"]
merged["mrr"] = merged["mrr"].fillna(0)

In [34]:
type(merged["mrr"].mean())

numpy.float64

In [24]:
val

,,date
session_id,item_id,
107,3370,2020-01-15 06:07:23.177
108,15639,2020-06-06 17:25:15.508
124,3126,2020-05-18 11:04:15.420
171,11806,2020-12-18 01:12:30.945
324,22049,2020-05-25 15:40:17.987
...,...,...
4439892,7623,2020-03-01 13:52:26.972
4439953,2179,2021-04-02 11:31:29.357
4439964,17216,2020-03-24 08:18:38.953
